# Using Vector Geometry in Cal-Adapt API

In [1]:
import requests 
import pprint

For getting climate data for an area, you can use the `g` query parameter to provide polygon geometry. Another way to provide geometry is to use polygons from the boundary datasets stored in Cal-Adapt API using the `ref` parameter. For a list of boundary datasets available through Cal-Adapt refer to the [documentation](https://berkeley-gif.github.io/caladapt-docs/data-catalog.html#vector-data).

In the cell below we are querying the `counties` resource to get a reference to the county polygon intersected by a point.

In [2]:
api = 'http://api.cal-adapt.org/api'
# Name of vector resource in API.
resource = 'counties'
# Variable for storing reference to data returned by the API
data = None
# HTTP header
headers = {'ContentType': 'json'}

# Point location in geojson format
lat = 38.5938
lng = -121.4687
point = '{"type":"Point","coordinates":[' + str(lng) + ',' + str(lat) + ']}'

# Params
params = {'intersects': point, 'srs': 4326, 'precision': 4}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())

# Construct url
url = api + '/' + resource + '/?' + params_str
print(url)
print()

# Get county geometry
response = requests.get(url, headers=headers)
if response.status_code == requests.codes.ok:
    data = response.json()
    pprint.pprint(data)

http://api.cal-adapt.org/api/counties/?intersects={"type":"Point","coordinates":[-121.4687,38.5938]}&srs=4326&precision=4

{'count': 1,
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[[-121.5216290001256,
                                               38.73641094745772],
                                              [-121.506266022973,
                                               38.73575192290427],
                                              [-121.4903329386042,
                                               38.73504292413254],
                                              [-121.4900559911822,
                                               38.73500398101285],
                                              [-121.4894260486141,
                                               38.73496403506602],
                                              [-121.4843870094908,
                                               3

Format a Uniform Resource Identifier (URI) for the polygon

In [3]:
features = data['features']
polygon = features[0]
polygon_uri = '/api/' + resource + '/' + str(polygon['id']) + '/'
print ('Uniform Resource Identifier for county:', polygon_uri)

Uniform Resource Identifier for county: /api/counties/34/


Get climate data for polygon

In [4]:
params = {
    'pagesize': 100,
    'ref': polygon_uri,
    'stat': 'mean'
}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/?' + params_str

# Make request
response = requests.get(url, headers=headers)

# It is a good idea to check there were no problems with the request.
if response.status_code == requests.codes.ok:
    data = response.json()
    # Get a list of Raster Stores from results property of data object
    results = data['results']
    print('Tiemseries for the this polygon:')
    # Iterate through the list and print the event and image property of each Raster Store
    for item in results:
        print('year:', item['event'], 'value:', item['image'], item['units'])

Tiemseries for the this polygon:
year: 2006-01-01 value: 297.9562661082474 K
year: 2007-01-01 value: 297.89267880154637 K
year: 2008-01-01 value: 297.9230025773196 K
year: 2009-01-01 value: 297.1394571520619 K
year: 2010-01-01 value: 298.71532699742266 K
year: 2011-01-01 value: 297.8216212951031 K
year: 2012-01-01 value: 297.9773477770619 K
year: 2013-01-01 value: 298.4366543170103 K
year: 2014-01-01 value: 298.0299210695876 K
year: 2015-01-01 value: 297.03950547680415 K
year: 2016-01-01 value: 297.5677351804124 K
year: 2017-01-01 value: 297.77386034149487 K
year: 2018-01-01 value: 296.8478576030928 K
year: 2019-01-01 value: 297.8725032216495 K
year: 2020-01-01 value: 296.5120006443299 K
year: 2021-01-01 value: 296.9846568943299 K
year: 2022-01-01 value: 297.72881765463916 K
year: 2023-01-01 value: 298.0693057345361 K
year: 2024-01-01 value: 298.06904397551546 K
year: 2025-01-01 value: 297.5053962628866 K
year: 2026-01-01 value: 297.76848421391753 K
year: 2027-01-01 value: 298.39324259

### Your turn

1. Get climate data for a different area. Change the `resource` type and/or change coordinates for `point` location